# Basic MBert

In [ ]:
class MBert(nn.Module):
    def __init__(self, model_path, model, hidden_size):
        super(Toxic_cls, self).__init__()
        self.model_path = model_path
        self.hidden_size = hidden_size
        self.model = model.from_pretrained(model_path, output_hidden_states=True, output_attentions=True)
        self.label_num = 2
        self.fc = nn.Linear(self.hidden_size, self.label_num)
    
    def forward(self, inputs, masks):
        last_hidden_state, pooler_output, hidden_states, attentions = self.model(input_ids=inputs, attention_mask = masks)
        fc_output = self.fc(pooler_output)

        return fc_output, attentions

# MBert with a pooler output that contains information from all hidden states

In [ ]:
class ModMBert(nn.Module):
    def __init__(self, model_path, model, hidden_size):
        super(Toxic_BERTMod, self).__init__()
        self.model_path = model_path
        self.hidden_size = hidden_size
        self.model = model.from_pretrained(model_path, output_hidden_states=True, output_attentions=True)
        self.label_num = 2
        self.first_fc = nn.Linear(9216,self.hidden_size)
        self.fc = nn.Linear(self.hidden_size, self.label_num)
    
    def forward(self, inputs, masks):
        last_hidden_state, pooler_output, hidden_states, attentions = self.model(input_ids=inputs, attention_mask = masks)

        out = torch.cat(tuple([hidden_states[i] for i in range(len(hidden_states)-1,0,-1)]), dim=-1)

        masks = masks.unsqueeze(2)

        out = torch.mul(out, masks)

        out = out[:, 0, :]

        out = self.first_fc(out)

        fc_output = self.fc(out)

        return fc_output, attentions